In [1]:
import matplotlib.pyplot as plt
import scipy.integrate
import torch
import sys  
sys.path.insert(0, '..')

from hnn.simulation import get_dataset, get_field, get_trajectory, get_vector_field, integrate_model
from hnn.train import train

In [2]:
import argparse

def get_args():
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument('--input_dim', default=2, type=int, help='dimensionality of input tensor')
    parser.add_argument('--hidden_dim', default=200, type=int, help='hidden dimension of mlp')
    parser.add_argument('--learn_rate', default=1e-3, type=float, help='learning rate')
    parser.add_argument('--total_steps', default=2000, type=int, help='number of gradient steps')
    parser.add_argument('--name', default='pend', type=str, help='only one option right now')
    parser.add_argument('--field_type', default='solenoidal', type=str, help='type of vector field to learn')
    parser.set_defaults(feature=True)
    return parser.parse_args()

In [3]:
args = get_args()
model, stats = train(args)

/Users/kristinlindquist/development/pinns/src/hnn/../hnn/train.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(data["x"], requires_grad=True, dtype=torch.float32)
/Users/kristinlindquist/development/pinns/src/hnn/../hnn/train.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_x = torch.tensor(data["test_x"], requires_grad=True, dtype=torch.float32)
/Users/kristinlindquist/development/pinns/src/hnn/../hnn/train.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dxdt = torch.tensor(data["dx"], requires_grad=Tr

X has grad? False
X has grad? False
step 0, train_loss 6.1798e+00, test_loss 6.6746e+00
X has grad? True
X has grad? False
step 1, train_loss 5.7570e+00, test_loss 6.4878e+00
X has grad? True
X has grad? False
step 2, train_loss 5.4049e+00, test_loss 6.2779e+00
X has grad? True
X has grad? False
step 3, train_loss 5.0936e+00, test_loss 6.0322e+00
X has grad? True
X has grad? False
step 4, train_loss 4.8002e+00, test_loss 5.7505e+00
X has grad? True
X has grad? False
step 5, train_loss 4.5122e+00, test_loss 5.4414e+00
X has grad? True
X has grad? False
step 6, train_loss 4.2255e+00, test_loss 5.1170e+00
X has grad? True
X has grad? False
step 7, train_loss 3.9413e+00, test_loss 4.7891e+00
X has grad? True
X has grad? False
step 8, train_loss 3.6622e+00, test_loss 4.4677e+00
X has grad? True
X has grad? False
step 9, train_loss 3.3908e+00, test_loss 4.1599e+00
X has grad? True
X has grad? False
step 10, train_loss 3.1281e+00, test_loss 3.8700e+00
X has grad? True
X has grad? False
step 1

In [4]:
GRID_SIZE=10
R = 2.5
t_span=torch.tensor([0, 28])
LINE_SEGMENTS = 10
ARROW_SCALE = 40
ARROW_WIDTH = 6e-3
LINE_WIDTH = 2

In [5]:

field = get_field(xmin=-R, xmax=R, ymin=-R, ymax=R, gridsize=GRID_SIZE)
vector_field = get_vector_field(model, xmin=-R, xmax=R, ymin=-R, ymax=R, gridsize=GRID_SIZE)

# integrate along those fields starting from point (1,0)
ivp = integrate_model(
        model,
        t_span,
        y0=torch.tensor([2.1, 0]),
        t_eval=torch.linspace(t_span[0], t_span[1], 1000),
        rtol=1e-12
)


/opt/homebrew/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


X has grad? False


RuntimeError: One of the differentiated Tensors does not require grad

In [ ]:
fig = plt.figure(figsize=(11.3, 3.2), facecolor='white', dpi=300)

# plot data
fig.add_subplot(1, 4, 2, frameon=True)
x, y, dx, dy, t = get_trajectory(t_span=[0,4], radius=2.1, y0=torch.tensor([2.1, 0]))
N = len(x)
point_colors = [(i/N, 0, 1-i/N) for i in range(N)]
plt.scatter(x,y, s=14, label='data', c=point_colors)
plt.quiver(
        field['x'][:,0],
        field['x'][:,1],
        field['dx'][:,0],
        field['dx'][:,1],
        cmap='gray_r',
        scale=ARROW_SCALE,
        width=ARROW_WIDTH,
        color=(.2,.2,.2)
)  
plt.xlabel("$q$", fontsize=14)
plt.ylabel("$p$", rotation=0, fontsize=14)
plt.title("Data", pad=10)

# plot HNN
fig.add_subplot(1, 4, 4, frameon=True)
plt.quiver(
        field['x'][:,0],
        field['x'][:,1],
        vector_field[:,0],
        vector_field[:,1],
        cmap='gray_r',
        scale=ARROW_SCALE,
        width=ARROW_WIDTH,
        color=(.5,.5,.5)
)

for i, l in enumerate(torch.tensor_split(torch.tensor(ivp['y'].T), LINE_SEGMENTS)):
        color = (float(i)/LINE_SEGMENTS, 0, 1-float(i)/LINE_SEGMENTS)
        plt.plot(l[:,0], l[:,1], color=color, linewidth=LINE_WIDTH)

plt.xlabel("$q$", fontsize=14)
plt.ylabel("$p$", rotation=0, fontsize=14)
plt.title("Hamiltonian NN", pad=10)
plt.tight_layout() ; plt.show()